### 🎯 任務說明：將 議論文 改寫為 Chain-of-Thought（CoT）推理過程

**原始任務：** 輸入一個議題生成一篇議論文。

#### ✅ CoT 改寫版本流程：
1. **第一階段（思考階段）**：請LLM以社經政文環(社會、經濟、政治、文化、環境)5個方面，思考3個同意論點和3個不同意論點。

2. **第二階段（產文階段）**：從6個論點中挑取最有說服力的4個論點，寫成一篇四段的議論文，第一、二段是我方的論點，第三、四段是都要有反方的論點而且駁論。

這就是典型的 Planning 模式應用：先拆解、後執行。

#### 1. 讀入Groq金鑰


In [1]:
import os
from google.colab import userdata

In [2]:
#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "openai/gpt-oss-120b"

In [3]:
!pip install aisuite[all]

### 2. 使用 AISuite 的準備

In [4]:
import aisuite as ai

In [5]:
provider_planner = "groq"
model_planner="openai/gpt-oss-120b"

provider_writer = "groq"
model_writer = "openai/gpt-oss-120b"

In [6]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="openai/gpt-oss-120b"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 打造二階段

In [7]:
system_planner = """
請用台灣習慣的中文回應:
你是一位專業的宏觀政策分析師，擅長從多方面（社會、經濟、政治、文化、環境）來剖析一個議題。
請針對使用者提供的議題，提供三個「同意」論點和三個「不同意」論點。
請務必在每個論點後面簡述它主要涉及的方面（例如：(經濟/環境)、(社會/文化)）。
用條列式清單呈現，保持中立且客觀的語氣。
"""
system_writer = """
請用台灣習慣的中文回應:
你是一位頂尖的學術議論文寫手，擅長使用清晰、邏輯嚴謹的台灣中文撰寫文章。
你的任務是將提供的論點組織成一篇四段式的議論文，並嚴格遵循以下格式要求來撰寫每一段：
1. 開頭： 每一段都必須以「在[社會/經濟/政治/文化/環境]方面，...」開頭，點明討論的方面。
2. 主體： 接著是該段的核心主題句和詳細論述。
3. 結尾： 以一句話的總結句結束該段論點。
文章結構如下：
一句表明我方立場
第一、二段
選擇我方最有說服力的論點撰寫。
第三、四段：
必須提出反方最具代表性的論點，並緊接著提供強有力的「駁論」。
"""

In [8]:
def argumentative_essay(prompt):
    # Step 1: CoT - 思考3個同意論點和3個不同意論點
    planning_prompt = f"使用者希望討論的議題是：『{prompt}』。請幫我想3個同意論點和3個不同意論點。"
    all_arguments = reply(system_planner, planning_prompt,
                          provider = provider_planner,
                          model = model_planner
                          )

    # Step 2: 選出最有說服力的5個論點，寫成議論文
    generation_prompt = f"這是根據議題『{prompt}』所產生的所有六個論點：{all_arguments} 請從中挑取最有說服力的4個論點，寫成一篇四段的議論文。第一、二段是我方的論點；第三、四段都要有反方的論點並緊接著提供駁論。"
    final_post = reply(system_writer, generation_prompt,
                       provider = provider_writer,
                       model = model_writer
                       )

    return all_arguments, final_post

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [9]:
!pip install gradio

In [10]:
import gradio as gr

In [11]:
with gr.Blocks() as demo:
    gr.Markdown("### 結構化議論文生成器 (CoT 論證模式)")
    gr.Markdown("請輸入一個您想討論的**公共議題**，AI 將會從社會、經濟、政治、文化、環境五個方面，為您生成正反論點，並寫成一篇具備駁論的三段式議論文。")

    user_input = gr.Textbox(label="請輸入您想討論的議題")

    btn = gr.Button("分析與寫作")

    with gr.Row():
        out1 = gr.Textbox(label="全方位論點清單 (正反共六項，標註方面)")

        out2 = gr.Textbox(label="最終四段式議論文")

    btn.click(argumentative_essay, inputs=[user_input], outputs=[out1, out2])

In [12]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://00dd45f0c2685137d4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://00dd45f0c2685137d4.gradio.live
